## Solve for Vehicle Routing Problem (VRP) or Traveling Salesman Problem (TSP)

In [ ]:
%pip install ortools ipyleaflet geopandas pyproj shapely

In [ ]:
site_path = "sample_data/"
img_path = site_path + "random_oilpalm.tif"
crowns_path = site_path + "crowns_exported.gpkg"
overlay_img_path = site_path + "overlay_rgb.png"

In [ ]:
# --- Imports ---
import rasterio
import asyncio
import geopandas as gpd
import numpy as np
from shapely.geometry import box, mapping, shape
from shapely.ops import transform
import pyproj
from pyproj import Geod
from PIL import Image
from ipyleaflet import (
    Map, GeoJSON, Marker, Popup, LayerGroup, DivIcon, AwesomeIcon, Icon,
    basemap_to_tiles, basemaps, ImageOverlay, LayersControl, TileLayer,
    WidgetControl, MarkerCluster, Polyline
)
from ipywidgets import HTML, Layout, Button
# --- Save RGB overlay as PNG from GeoTIFF ---
def save_rgb_overlay_as_png(img_path, output_img="overlay.png"):
    with rasterio.open(img_path) as src:
        img = src.read()
        bounds = src.bounds
        crs = src.crs
        
        if img.shape[0] >= 3:
            rgb = np.stack([img[0], img[1], img[2]], axis=-1)
        else:
            raise ValueError("Need at least 3 bands for RGB")
        
        # Normalize for better display
        rgb = rgb.astype(np.float32)
        rgb_min, rgb_max = np.percentile(rgb[rgb > 0], (2, 98))
        rgb = np.clip((rgb - rgb_min) / (rgb_max - rgb_min), 0, 1)
        rgb = (rgb * 255).astype(np.uint8)
        
        image = Image.fromarray(rgb)
        image.save(output_img)
        
        return output_img, bounds, crs

overlay_img_path, utm_bounds, utm_crs = save_rgb_overlay_as_png(img_path)

# --- Convert UTM bounds to WGS84 ---
def reproject_bounds_to_wgs84(bounds, src_crs):
    project = pyproj.Transformer.from_crs(src_crs, "EPSG:4326", always_xy=True).transform
    return transform(project, box(*bounds)).bounds

wgs84_bounds = reproject_bounds_to_wgs84(utm_bounds, utm_crs)
lon_min, lat_min, lon_max, lat_max = wgs84_bounds

# --- Load crowns and reproject to WGS84 ---
crowns = gpd.read_file(crowns_path)
if crowns.crs != "EPSG:4326":
    crowns = crowns.to_crs("EPSG:4326")

In [ ]:
# --- USER INPUT ---
# Home/origin location (lat, lon)
home = (1.954826, 103.214203)

# List of crown IDs to visit
visit_ids = [27, 28, 30, 31, 79]

# Drone parameters
drone_speed = 1.5        # meters per second
max_flight_time = 480.0  # seconds
max_distance = drone_speed * max_flight_time  # meters

In [ ]:
import numpy as np
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from pyproj import Transformer

# --- 1. Load crowns and extract coordinates ---
coords = {}
for idx, row in crowns.iterrows():
    # Extract centroid coordinates from the polygon
    coords[idx] = (row.geometry.centroid.y, row.geometry.centroid.x)

# Build ordered list of nodes: depot=0, then each visit_id
node_ids = [0] + visit_ids  # 0 represents home/depot
node_coords = {0: home}     # Add home coordinates as node 0

# Add coordinates for visit points
for vid in visit_ids:
    if vid in coords:
        node_coords[vid] = coords[vid]
    else:
        print(f"Warning: ID {vid} not found in coordinates dictionary")
        
# --- 2. Compute distance matrix (UTM -> Euclidean in meters) ---
# Transformer to UTM (choose zone automatically)
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3857", always_xy=True)  # Web Mercator
n = len(node_ids)
dist_matrix = np.zeros((n, n), dtype=int)
for i, id_i in enumerate(node_ids):
    x_i, y_i = transformer.transform(node_coords[id_i][1], node_coords[id_i][0])
    for j, id_j in enumerate(node_ids):
        x_j, y_j = transformer.transform(node_coords[id_j][1], node_coords[id_j][0])
        dist_matrix[i, j] = int(np.hypot(x_i - x_j, y_i - y_j))

print(dist_matrix)

# --- 3. Set up OR-Tools routing for Orienteering ---
manager = pywrapcp.RoutingIndexManager(
    len(dist_matrix), # number of nodes
    1,                # number of vehicles
    0                 # depot node index
)
routing = pywrapcp.RoutingModel(manager)

# Distance callback
def distance_callback(i, j):
    return dist_matrix[manager.IndexToNode(i)][manager.IndexToNode(j)]
transit_cb = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_cb)

# Add distance dimension with cap
routing.AddDimension(
    transit_cb,
    slack_max=0,
    capacity=int(max_distance),
    fix_start_cumul_to_zero=True,
    name='Distance'
)

# Allow dropping nodes with penalty (1 per node)
penalty = 1000  # cost for skipping a node
for node in range(1, n):
    routing.AddDisjunction([manager.NodeToIndex(node)], penalty)

# Search parameters
search_params = pywrapcp.DefaultRoutingSearchParameters()
search_params.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
search_params.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
search_params.time_limit.seconds = 10

solution = routing.SolveWithParameters(search_params)

# --- 4. Extract the optimized route ---
route_ids = []
if solution:
    idx = routing.Start(0)
    while not routing.IsEnd(idx):
        node = manager.IndexToNode(idx)
        route_ids.append(node_ids[node])
        idx = solution.Value(routing.NextVar(idx))
    route_ids.append(node_ids[manager.IndexToNode(idx)])  # return home
    total_dist = solution.ObjectiveValue()
    print("Optimized route (in visit order):", route_ids)
    print(f"Total distance: {total_dist} m  (max allowed {max_distance} m)")
else:
    print("No feasible route found within flight time.")

In [ ]:
crowns["id"] = crowns.index
crowns_json = crowns.__geo_interface__

for i, feat in enumerate(crowns_json["features"]):
    feat["id"] = i

# --- Initialize Map ---
center_lat = (lat_min + lat_max) / 2
center_lon = (lon_min + lon_max) / 2

m = Map(center=(center_lat, center_lon), zoom=18, max_zoom=20)
m.layout.width = '100%'
m.layout.height = '900px'

# --- Base layers ---
google_sat = TileLayer(
    url="http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attribution="Google Satellite",
    name="Google Satellite",
    max_zoom=20
)
m.add_layer(google_sat)

esri_sat = basemap_to_tiles(basemaps.Esri.WorldImagery)
esri_sat.name = "ESRI Imagery"
m.add_layer(esri_sat)

# --- Overlay image ---
overlay = ImageOverlay(
    url=overlay_img_path,
    bounds=[(lat_min, lon_min), (lat_max, lon_max)],
    opacity=0.6,
    name="Oil Palm RGB"
)
m.add_layer(overlay)

# Add LayersControl after adding base and overlay layers
m.add_control(LayersControl(position="topright"))

# --- Debug output widget (outside the map) ---
debug_output = HTML(
    value="Debug info will appear here and in console",
    layout=Layout(width='100%', height='50px', border='1px solid gray', padding='5px')
)

# --- Crown ID label layer ---
crown_id_layer = LayerGroup(name="Crown IDs")
m.add_layer(crown_id_layer)

for feature in crowns_json['features']:
    crown_id = feature['id']
    geometry = feature['geometry']
    if geometry['type'] == 'Polygon':
        polygon = shape(geometry)
        centroid = polygon.centroid
        icon = DivIcon(
                icon_size=(20, 20),
                icon_anchor=(10, 10),
            )
        if crown_id in route_ids:
            if crown_id == route_ids[-1]:
                icon.html = f"<div style='color: #ffffff; font-size: 12px; text-align: center; text-shadow: -1px -1px 0 black, 1px -1px 0 black, -1px 1px 0 black, 1px 1px 0 black;'>{crown_id}</div>"
            else:
                icon.html=f"<div style='color: darkblue; font-size: 12px; font-weight: bold; text-align: center; text-shadow: -1px -1px 0 blue, 1px -1px 0 blue, -1px 1px 0 blue, 1px 1px 0 blue;'>{crown_id}</div>"
        else:
            icon.html = f"<div style='color: #ffffff; font-size: 12px; text-align: center; text-shadow: -1px -1px 0 black, 1px -1px 0 black, -1px 1px 0 black, 1px 1px 0 black;'>{crown_id}</div>"
        marker = Marker(location=(centroid.y, centroid.x), icon=icon, draggable=False)
        crown_id_layer.add_layer(marker)

marker_layer = MarkerCluster(name="Crown Id")
m.add_layer(marker_layer)

# Create GeoJSON with appropriate settings to prevent layer issues
crown_layer = GeoJSON(
    data=crowns_json,
    name="Tree Crowns",
    hover_style={"fillColor": "red", "fillOpacity": 0.5},
    style={"fillOpacity": 0.3, "color": "yellow", "weight": 0.5},
    click_events=True 
)

# Register event handler properly
m.add_layer(crown_layer)

def handle_mouse_move(**kwargs):
    try:
        # Extract lat/lon from the event data
        lat = kwargs.get('coordinates', [0, 0])[0]
        lon = kwargs.get('coordinates', [0, 0])[1]
        
        # Format and display in the existing debug_output widget
        debug_output.value = f"Hover position - Lat: {lat:.6f}, Lon: {lon:.6f}"
        
        # No print() statement, so nothing goes to the terminal
    except Exception as e:
        # Only update the debug_output with error information
        debug_output.value = f"Error in mouse move handler: {e}"

# m.on_interaction(handle_mouse_move)

# Display route
# --- Animation Setup ---
frame_delay = 0.1  # seconds per frame
speed_up_factor = 5.0
step_distance = drone_speed * frame_delay * speed_up_factor

# Original coarse route
route_coords = [node_coords[vid] for vid in route_ids]

def interpolate_segment(p1, p2, step_dist):
    """
    Returns a list of interpolated points between p1 and p2 using step_dist spacing.
    """
    geod = Geod(ellps="WGS84")
    lon1, lat1 = p1[1], p1[0]
    lon2, lat2 = p2[1], p2[0]
    az12, az21, total_dist = geod.inv(lon1, lat1, lon2, lat2)
    
    n_steps = max(int(total_dist // step_dist), 1)
    interpolated_points = []
    
    for i in range(n_steps + 1):
        dist = (i / n_steps) * total_dist
        lon, lat, _ = geod.fwd(lon1, lat1, az12, dist)
        interpolated_points.append((lat, lon))
    
    return interpolated_points

def generate_interpolated_route(route_coords, step_dist=2.0):
    """
    Generate interpolated route with consistent spacing (in meters).
    """
    full_route = []
    for i in range(len(route_coords) - 1):
        segment = interpolate_segment(route_coords[i], route_coords[i+1], step_dist)
        # Avoid duplicating points
        if i > 0:
            segment = segment[1:]
        full_route.extend(segment)
    return full_route

# Interpolated smooth route
smooth_route = generate_interpolated_route(route_coords, step_dist=step_distance)

# Quadcopter-style icon (customize with a better image if desired)
quad_icon = AwesomeIcon(
    name='fighter-jet',
    marker_color="black",
    icon_color='white',
    icon_size=[30, 30],
    icon_anchor=[15, 15]
)

# --- Animated Marker and Polyline in a LayerGroup ---
animated_marker = Marker(location=route_coords[0], icon=quad_icon)
animated_line = Polyline(locations=[route_coords[0]], color="cyan", fill=False, weight=4)

animation_layer = LayerGroup(name="Drone Route")
animation_layer.add_layer(animated_marker)
animation_layer.add_layer(animated_line)
m.add_layer(animation_layer)

# --- Animate with variable delay ---
async def animate_route(marker, polyline, route, delay=0.1):
    for coord in route[1:]:
        await asyncio.sleep(delay)
        marker.location = coord
        polyline.locations = polyline.locations + [coord]

# --- Start Button with Styling ---
start_button = Button(
    description="Start Drone Animation",
    layout=Layout(width='220px', height='40px', margin='10px 0px 10px 10px'),
    style={'button_color': "#1fb49b", 'font_weight': 'bold', 'font_size': '14px'}
)

def on_start_click(b):
    start_button.disabled = False
    loop = asyncio.get_event_loop()
    loop.create_task(animate_route(animated_marker, animated_line, smooth_route, delay=frame_delay))

start_button.on_click(on_start_click)

# Display the start button below the map
display(start_button)

# Display the map and debug area
display(m)
display(debug_output)
